<a href="https://colab.research.google.com/github/bikash119/learn_tensorflow/blob/main/learn_tf_009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning Part 2: Fine-Tuning

### What we will cover
1. Introduce fine-tuning, a transfer learning approach to modify pre-trained layers to be more suited for our data.
2. Using the keras Functional API
3. Smaller dataset of faster experimentation
4. Data Augmenetation ( how to make our training dataset more diverse without adding more data).
5. Running a series of model experiments using our Food Vision data
    1. Model0: A transfer learning model using Keras functional api
    2. Model1: A feature extraction transfer learning model using 1% of the data with data augmentation
    3. Model2: A feature extraction transfer learning model using 10% of the data with data augmentation.
    4. Model3: A fine-tuned transfer learning model on 10% of data.
    5. Model4: A fine-tuned transfer learning model on 100% of data.
6. Introduce model checkpoints to save intermediate model results
7. Compare the model experiments using tensorboard.




In [1]:
import datetime
print(f" Notebook last run (end-to-end) {datetime.datetime.now()}")

 Notebook last run (end-to-end) 2023-08-02 23:34:54.131200


In [2]:
import tensorflow as tf
print(f" Tensorflow version : {tf.__version__}")

 Tensorflow version : 2.12.0


In [35]:
%%writefile helper_functions.py
import matplotlib.pyplot as plt

def plot_loss_curve(history):
  """
    Plots the loss and accuracy curve for training and validation metrics
    Args:
      history(A History object):

  """
  epochs= range(len(history.history["loss"]))
  loss= history.history["loss"]
  val_loss= history.history["accuracy"]

  plt.plot(loss,epochs,label="Train loss")
  plt.plot(val_loss,epochs,label="Validation loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  ## Plot Accuracy
  plt.figure()
  accuracy= history.history["accuracy"]
  val_accuracy= history.history["val_accuracy"]
  plt.title("Accuracy")
  plt.xlabel("Epochs")
  plt.legend();



Writing helper_functions.py


In [8]:
from helper_functions import plot_loss_curve

In [9]:
## Download data and unzip it
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip



--2023-08-03 00:08:30--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 74.125.137.128, 142.250.101.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   127MB/s    in 1.3s    

2023-08-03 00:08:31 (127 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [36]:
%%writefile -a helper_functions.py

import zipfile
import os
def unzip_data(zipped_file,remove_src_zip=True):
  """
    Takes a zipped file and unzips it.Removes the source zipfile.
  """
  zip_ref= zipfile.ZipFile(zipped_file,"r")
  zip_ref.extractall()
  zip_ref.close()
  os.remove(zipped_file)

Appending to helper_functions.py


In [15]:
unzip_data("/content/10_food_classes_10_percent.zip")

In [16]:
!rm -r "/content/__MACOSX"

In [26]:
%less helper_functions.py

In [27]:
os.walk?

In [6]:
data_dir= "/content/10_food_classes_10_percent"

In [37]:
%%writefile -a helper_functions.py

def walk_through_dir(data_dir):
  for dirpath,dirnames,filenames in os.walk(data_dir):
    print(f" There are {len(filenames)} images and {len(dirnames)} folder at {dirpath}")



Appending to helper_functions.py


In [7]:
import sys
sys.path.append("/content")
from helper_functions import walk_through_dir
walk_through_dir(data_dir)


 There are 0 images and 2 folder at /content/10_food_classes_10_percent
 There are 0 images and 10 folder at /content/10_food_classes_10_percent/test
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/sushi
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/chicken_curry
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/ramen
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/grilled_salmon
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/chicken_wings
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/ice_cream
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/fried_rice
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/hamburger
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/steak
 There are 250 images and 0 folder at /content/

In [39]:
%less helper_functions.py

In [50]:


unzip_data("xyz")

FileNotFoundError: ignored